#Uploads and Imports

In [ ]:
# uploading files
# source: https://towardsdatascience.com/3-ways-to-load-csv-files-into-colab-7c14fcbdcb92
from google.colab import files
uploaded = files.upload()

Saving imdb-ratings-basics.csv to imdb-ratings-basics.csv


In [ ]:
# uploading files
# source: https://towardsdatascience.com/3-ways-to-load-csv-files-into-colab-7c14fcbdcb92
from google.colab import files
uploaded = files.upload()

Saving im.db to im.db


In [ ]:
import numpy as np
import pandas as pd
import sqlite3

In [ ]:
df_imdb = pd.read_csv('imdb-ratings-basics.csv')

In [ ]:
conn = sqlite3.connect('im.db')
cur = conn.cursor()

#Previews and Dropping Junk Columns

##IMDB

In [ ]:
df_imdb.head(3)

,Unnamed: 0,movie_id,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,main_genre,subgenre_1,subgenre_2
0,0,tt0063540,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77,Action,Crime,Drama
1,1,tt0066787,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43,Biography,Drama,NaN
2,2,tt0069049,The Other Side of the Wind,2018,122.0,Drama,6.9,4517,Drama,NaN,NaN


In [ ]:
df_imdb.drop(columns=(['Unnamed: 0']), inplace=True)

In [ ]:
df_imdb.head(3)

,movie_id,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,main_genre,subgenre_1,subgenre_2
0,tt0063540,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77,Action,Crime,Drama
1,tt0066787,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43,Biography,Drama,NaN
2,tt0069049,The Other Side of the Wind,2018,122.0,Drama,6.9,4517,Drama,NaN,NaN


In [ ]:
df_imdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66236 entries, 0 to 66235
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   movie_id         66236 non-null  object 
 1   original_title   66236 non-null  object 
 2   start_year       66236 non-null  int64  
 3   runtime_minutes  66236 non-null  float64
 4   genres           65720 non-null  object 
 5   averagerating    66236 non-null  float64
 6   numvotes         66236 non-null  int64  
 7   main_genre       65720 non-null  object 
 8   subgenre_1       34877 non-null  object 
 9   subgenre_2       17840 non-null  object 
dtypes: float64(2), int64(2), object(6)
memory usage: 5.1+ MB


#Now to add director data...

##Joining directors and persons

## Checking the number of directors to compare to number of movies.

In [ ]:
pd.read_sql(
    """
SELECT COUNT(DISTINCT movie_id) as num_movies
FROM directors
""", conn
)

,num_movies
0,140417


In [ ]:
pd.read_sql(
    """
SELECT COUNT(DISTINCT person_id) as num_directors
FROM directors
""", conn
)

,num_directors
0,109253


###Joining `directors` and `persons` tables to get director data

In [ ]:
df_directors_persons = pd.read_sql(
    """
    SELECT movie_id, directors.person_id, primary_name, birth_year, death_year, primary_profession
    FROM persons
    JOIN directors
    ON persons.person_id = directors.person_id
    GROUP BY movie_id
    ORDER BY primary_name;
    """, conn
)

In [ ]:
df_directors_persons.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140416 entries, 0 to 140415
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   movie_id            140416 non-null  object 
 1   person_id           140416 non-null  object 
 2   primary_name        140416 non-null  object 
 3   birth_year          31232 non-null   float64
 4   death_year          875 non-null     float64
 5   primary_profession  139862 non-null  object 
dtypes: float64(2), object(4)
memory usage: 6.4+ MB


In [ ]:
df_directors_persons.head()

,movie_id,person_id,primary_name,birth_year,death_year,primary_profession
0,tt3433162,nm2764095,'Atlas' Ramachandran,NaN,NaN,"actor,director"
1,tt6002234,nm8389925,0010x0010,NaN,NaN,"director,editor,composer"
2,tt6290518,nm8617424,A Fernandez,NaN,NaN,director
3,tt6158688,nm8518087,A R Ravindra,NaN,NaN,director
4,tt4795624,nm3499396,A'Ali de Sousa,NaN,NaN,"actor,producer,writer"


###Merging our previous IMDB dataframe (made by joining the  `movie_basics` table with `movie_ratings` table) with our directors dataframe (made by joining the `directors` table with the `persons` table)

In [ ]:
df_directors_movies_imdb = pd.merge(df_imdb, df_directors_persons, on='movie_id', how='left')

In [ ]:
df_directors_movies_imdb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66236 entries, 0 to 66235
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   movie_id            66236 non-null  object 
 1   original_title      66236 non-null  object 
 2   start_year          66236 non-null  int64  
 3   runtime_minutes     66236 non-null  float64
 4   genres              65720 non-null  object 
 5   averagerating       66236 non-null  float64
 6   numvotes            66236 non-null  int64  
 7   main_genre          65720 non-null  object 
 8   subgenre_1          34877 non-null  object 
 9   subgenre_2          17840 non-null  object 
 10  person_id           65760 non-null  object 
 11  primary_name        65760 non-null  object 
 12  birth_year          20644 non-null  float64
 13  death_year          536 non-null    float64
 14  primary_profession  65704 non-null  object 
dtypes: float64(4), int64(2), object(9)
memory usage: 8.1+

###Creating a 2010-2018 IMDB dataframe

In [ ]:
# making a copy

df_directors_movies_imdb_copy = df_directors_movies_imdb.copy()

In [ ]:
df_directors_movies_imdb_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66236 entries, 0 to 66235
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   movie_id            66236 non-null  object 
 1   original_title      66236 non-null  object 
 2   start_year          66236 non-null  int64  
 3   runtime_minutes     66236 non-null  float64
 4   genres              65720 non-null  object 
 5   averagerating       66236 non-null  float64
 6   numvotes            66236 non-null  int64  
 7   main_genre          65720 non-null  object 
 8   subgenre_1          34877 non-null  object 
 9   subgenre_2          17840 non-null  object 
 10  person_id           65760 non-null  object 
 11  primary_name        65760 non-null  object 
 12  birth_year          20644 non-null  float64
 13  death_year          536 non-null    float64
 14  primary_profession  65704 non-null  object 
dtypes: float64(4), int64(2), object(9)
memory usage: 8.1+

In [ ]:
df_imdb_2010_2018 = df_directors_movies_imdb_copy[(df_directors_movies_imdb_copy['start_year'] > 2009) & (df_directors_movies_imdb_copy['start_year'] < 2019)]

In [ ]:
df_imdb_2010_2018.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64425 entries, 0 to 66228
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   movie_id            64425 non-null  object 
 1   original_title      64425 non-null  object 
 2   start_year          64425 non-null  int64  
 3   runtime_minutes     64425 non-null  float64
 4   genres              63916 non-null  object 
 5   averagerating       64425 non-null  float64
 6   numvotes            64425 non-null  int64  
 7   main_genre          63916 non-null  object 
 8   subgenre_1          34068 non-null  object 
 9   subgenre_2          17479 non-null  object 
 10  person_id           63964 non-null  object 
 11  primary_name        63964 non-null  object 
 12  birth_year          20081 non-null  float64
 13  death_year          533 non-null    float64
 14  primary_profession  63922 non-null  object 
dtypes: float64(4), int64(2), object(9)
memory usage: 7.9+

In [ ]:
df_imdb_2010_2018.head(3)

,movie_id,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,main_genre,subgenre_1,subgenre_2,person_id,primary_name,birth_year,death_year,primary_profession
0,tt0063540,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77,Action,Crime,Drama,nm0712540,Harnam Singh Rawail,1921.0,2004.0,"director,writer,producer"
2,tt0069049,The Other Side of the Wind,2018,122.0,Drama,6.9,4517,Drama,NaN,NaN,nm0000080,Orson Welles,1915.0,1985.0,"actor,director,writer"
3,tt0100275,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119,Comedy,Drama,Fantasy,nm0749914,Raoul Ruiz,1941.0,2011.0,"director,writer,producer"


In [ ]:
df_imdb_2010_2018['primary_name'].value_counts()

Nayato Fio Nuala         28
Michael Fredianelli      26
David DeCoteau           25
William Winckler         23
Sergey A.                22
                         ..
William Scherer           1
Basilio Martín Patino     1
Damir Catic               1
George Vonknorring        1
Mark Goldman              1
Name: primary_name, Length: 43563, dtype: int64

##Exporting csv's

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
path = '/content/drive/My Drive/df_directors_movies_imdb.csv'
with open(path, 'w', encoding = 'utf-8-sig') as f:
  df_directors_movies_imdb.to_csv(f)

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
path = '/content/drive/My Drive/df_directors_persons.csv'
with open(path, 'w', encoding = 'utf-8-sig') as f:
  df_directors_persons2.to_csv(f)

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
path = '/content/drive/My Drive/df_imdb_2010_2018.csv'
with open(path, 'w', encoding = 'utf-8-sig') as f:
  df_imdb_2010_2018.to_csv(f)